In [28]:
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

In [29]:
with open('intents.json', 'r') as file:
    data = json.load(file)

In [30]:
tags = []
patterns = []
for intent in data['intents']:
  for pattern in intent['patterns']:
    patterns.append(pattern)
    tags.append(intent['tag'])


In [31]:
# Encoding using LabelEncoder
encode = LabelEncoder()
encoded_tags = encode.fit_transform(tags)

In [32]:
encoded_tags

array([3, 3, 3, 3, 3, 3, 7, 7, 7, 7, 7, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 4,
       4, 4, 4, 4, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 8, 8, 8,
       8, 8])

In [39]:
# Vectorizing each sentence of patterns
vectorizer = TfidfVectorizer()
vectorized_patterns = vectorizer.fit_transform(patterns)

In [57]:
# Building model
model = Sequential()
model.add(Dense(128, input_dim=vectorized_patterns.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(encode.classes_), activation='softmax'))

model.compile(metrics=['accuracy'], loss='sparse_categorical_crossentropy', optimizer='adam')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [93]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_23 (Dense)                     │ (None, 128)                 │          10,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 9)                   │             585 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 56,861 (222.12 KB)

 Trainable params: 18,953 (74.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 37,908 (148.08 KB)

In [59]:
model.fit(vectorized_patterns, encoded_tags, epochs=50)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2989 - loss: 2.1035 
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.4171 - loss: 2.0768 
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3216 - loss: 2.1000 
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3755 - loss: 2.0670 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4004 - loss: 2.0603 
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3610 - loss: 2.0379 
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3755 - loss: 2.0161 
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4896 - loss: 2.0221 
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4647 - loss: 2.0061 
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4937 - loss: 1.9697 
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6495 - loss: 1.9395 
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6141 - loss:

In [91]:
# Saving the model
import pickle
with open('model.pkl','wb') as f:
    pickle.dump(model,f)

In [ ]:
# Load the model
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [87]:
# Function to generate responses
def generate_responses(sentence):
  vectorized_sentence = vectorizer.transform([sentence]).toarray()
  model_prediction = model.predict(vectorized_sentence)
  tag = encode.inverse_transform([np.argmax(model_prediction)])
  intent = data['intents']
  for intent in data['intents']:
    if intent['tag'] == tag:
      response = np.random.choice(intent['responses'])
      return response

In [89]:
sentence = "i want to order some brocollies"
generate_responses(sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


'Sure! You can place an order on our website or through our app. Would you like me to guide you through the process?'